#참고 논문

http://www.koreascience.kr/article/JAKO202012764216667.pdf

##독립변수:기초금액, 추정가격, 예가 범위, 낙찰 하한율
##종속변수:낙찰금액

https://data.g2b.go.kr:1443/pt/pubdata/moveCntrwkBidPblancNdProgrsPop.do

data=공사 입찰공고 및 진행 내역(2022/1/1 ~2022/12/31) 자료를 이용함

#낙찰 하한가에 가장 근접하고 1순위 금액 사이의 금액을 예측하여 낙찰에 필요한 입찰금액을 예측하기 위한 기법을 제안

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl

# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'

# 경고 무시
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl

# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'

# 경고 무시
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
train =  pd.read_excel('/content/drive/MyDrive/입찰금액 예측모델/dataset/2018__2022.xlsx')

In [ ]:
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

In [ ]:
import xgboost

In [ ]:
from sklearn.model_selection import cross_val_score,cross_validate

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
train.columns

Index(['기초금액', '추정가격', '예가범위', '낙찰하한율', '낙찰금액'], dtype='object')

In [ ]:
train_feature = train.drop(['낙찰금액'], 1)
train_label = train['낙찰금액']

In [ ]:
train['기초금액']=train['기초금액'].astype('float32')

In [ ]:
train['추정가격']=train['추정가격'].astype('float32')

In [ ]:
train['예가범위']=train['예가범위'].astype('float32')

In [ ]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40380 entries, 0 to 40379
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기초금액    40380 non-null  float32
 1   추정가격    40380 non-null  float32
 2   예가범위    40380 non-null  float32
 3   낙찰하한율   40380 non-null  float64
 4   낙찰금액    40380 non-null  float64
dtypes: float32(3), float64(2)
memory usage: 1.1 MB
None


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_feature, train_label ,test_size=0.1)
xgb_model = XGBRegressor(objective ='reg:squarederror', learning_rate = 0.01,max_depth = 5, alpha = 10, n_estimators = 10000, num_boost_round=5000)
print(len(X_train), len(X_test))
xgb_model.fit(X_train,y_train)

36342 4038


XGBRegressor(alpha=10, learning_rate=0.01, max_depth=5, n_estimators=10000,
             num_boost_round=5000, objective='reg:squarederror')

In [ ]:
y_pred = xgb_model.predict(X_test)

In [ ]:
result = pd.DataFrame({'pred' : y_pred, 'real' : y_test})

result.head()

,pred,real
39790,465535680.0,799071042.0
919,92879696.0,95970392.0
3305,68460912.0,68953838.0
21986,160833856.0,189437857.0
6959,44834988.0,34176721.0


In [ ]:
score = xgb_model.score(X_train,y_train) 
print("Training score: ", score)

Training score:  0.9809573044318062


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [ ]:
lgbm_model = LGBMRegressor(n_estimators=30, learning_rate=0.12)
lgbm_model.fit(X_train, y_train)
train_error = mse(y_train, lgbm_model.predict(X_train))
test_error = mse(y_test, lgbm_model.predict(X_test))
print('Boosting test error:', test_error)
print('Boosting overfit ratio:', test_error/train_error)

Boosting test error: 2.78900729384583e+16
Boosting overfit ratio: 2.494833996635517


In [ ]:
def train_xgb(train_feature, train_label, num_split=3):
    preds=xgb_model.predict(X_test)
    return preds

In [ ]:
def train_lgbm(train_feature, train_label, test_feature, num_split=5):
    preds=lgbm_model.predict(X_test)
    return preds

In [ ]:
def train_xgb(train_feature, train_label, test_feature, num_split=3):
    preds=xgb_model.predict(X_test)
    return preds

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_feature, train_label, random_state=321)
X_train.shape, X_test.shape

((30285, 4), (10095, 4))

In [ ]:
lgbm_pred = train_lgbm(X_train, y_train, y_test)
xgb_pred = train_xgb(X_train, y_train, y_test)

In [ ]:
pred = lgbm_pred * 0.5 + xgb_pred * 0.5

In [ ]:
abs(pred - y_test).mean()

28404035.273121133